In [14]:
import cv2
import numpy as np
 
def make_points(img, line):   
    try:
        slope, intercept = line
    except TypeError:
        slope, intercept = 0.001, 0

    y1 = int(img.shape[0])
    y2 = int(y1*0.75)
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    return [[x1, y1, x2, y2]]

def moving_average(fit):
    N = 20
    if fit == 0:
        return fit
    fit -= fit / N;
    fit += fit / N;
    return fit
 
def average_slope_intercept(img, lines):
    left_fit    = []
    right_fit   = []
    
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                fit = np.polyfit((x1,x2), (y1,y2), 1)
                slope = fit[0]
                intercept = fit[1]
                if slope < 0:
                    left_fit.append((slope, intercept))
                else:
                    right_fit.append((slope, intercept))

    left_fit_average  = np.average(left_fit[-20:], axis=0)
    right_fit_average = np.average(right_fit[-30:], axis=0)
    left_line  = make_points(img, left_fit_average)
    right_line = make_points(img, right_fit_average)
    averaged_lines = [left_line, right_line]
    return averaged_lines
 
def display_lines(img, lines):
    line_image = np.zeros_like(img)
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
    return line_image
 
def roi(img):   
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    canny = cv2.Canny(gray, 50, 150)
    mask = np.zeros_like(canny)
 
    shape = np.array([[
    (250, height-20),
    (550, 570),
    (650, 570),
    (1000, height-20),]], np.int32)
 
    cv2.fillPoly(mask, shape, 255)
    masked_image = cv2.bitwise_and(canny, mask)
    return masked_image
 
cap = cv2.VideoCapture(r"C:\Users\Administrator\Desktop\programming\datasets\video\lane_test_01.mp4")
out = cv2.VideoWriter('output.avi', -1, 20.0, (640,480))
while(cap.isOpened()):
    width  = cap.get(3)
    height = cap.get(4)   
    _, frame = cap.read()
    
    cropped_canny = roi(frame)
    lines = cv2.HoughLinesP(cropped_canny, 1, np.pi/180, 20, np.array([]), minLineLength=20,maxLineGap=50)
    av_lines = average_slope_intercept(frame, lines)
    line_image = display_lines(frame, av_lines)
    combo_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1)

    cv2.imshow("cropped_canny", cropped_canny)
    cv2.imshow("result", combo_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
out.release()
cv2.destroyAllWindows()